# Data Validation

In the previous notebook, two pins were saved:

- City of Chicago - Business License Data (RAW): `chicago-business-license-data`
- ity of Chicago - Food Inspection Data (RAW): `chicago-food-inspection-data`

## Setup

In [1]:
import pandas as pd
import pandera as pa
import pins
from ydata_profiling import ProfileReport

In [2]:
pd.options.display.max_columns = 999

In [3]:
# Set up the board
board = pins.board_connect()
user_name = "sam.edwardes"

## Data set (1): Business License Data

<https://data.cityofchicago.org/Community-Economic-Development/Business-Licenses/r5kz-chrr>

In [4]:
pin_name = f"{user_name}/chicago-business-license-data-raw"
business_license_raw = board.pin_read(pin_name)
business_license_raw

,id,license_id,account_number,site_number,legal_name,doing_business_as_name,address,city,state,zip_code,ward,precinct,ward_precinct,police_district,license_code,license_description,business_activity_id,business_activity,license_number,application_type,application_created_date,application_requirements_complete,payment_date,conditional_approval,license_start_date,expiration_date,license_approved_for_issuance,date_issued,license_status,license_status_change_date,ssa,latitude,longitude,location
0,1000000-20020221,1000000,200001,1,MARK BOSTON,COLORS IN MOTION,6421 N DAMEN AVE,CHICAGO,IL,60645,50,28,50-28,24,1011,Home Repair,NaN,NaN,1000000,ISSUE,2000-06-19T00:00:00.000,2002-02-15T00:00:00.000,2002-02-15T00:00:00.000,N,2002-02-21T00:00:00.000,2002-11-15T00:00:00.000,2002-02-21T00:00:00.000,2002-02-22T00:00:00.000,AAI,NaN,NaN,41.998514,-87.680011,"\n, \n(41.99851437112669, -87.68001090539342)"
1,1000049-20010816,1162772,200068,1,ANTONIA CASTREJON,ILLUSIONS HAIR DESIGN,3800 W DIVERSEY AVE,CHICAGO,IL,60647,31,999,31-999,25,1010,Limited Business License,NaN,NaN,1000049,RENEW,NaN,2001-06-25T00:00:00.000,2001-08-20T00:00:00.000,N,2001-08-16T00:00:00.000,2002-08-15T00:00:00.000,2001-08-20T00:00:00.000,2002-04-30T00:00:00.000,AAI,NaN,NaN,41.931960,-87.722150,"\n, \n(41.931960332638006, -87.72215036594574)"
2,1000049-20020516,1233615,10141,2,"PEPE""S RETAIL MEATS, INC.",PEREZ MEXICAN FOOD,853-855 W RANDOLPH ST 1ST,CHICAGO,IL,60607,27,1,27-1,12,1006,Retail Food Establishment,775.0,Retail Sales of Perishable Foods,1000049,RENEW,NaN,2002-03-27T00:00:00.000,2002-04-17T00:00:00.000,N,2002-05-16T00:00:00.000,2003-05-15T00:00:00.000,2002-04-17T00:00:00.000,2002-04-18T00:00:00.000,AAI,NaN,NaN,41.884261,-87.649534,"\n, \n(41.88426142200001, -87.6495341312589)"
3,1000049-20020816,1265665,200068,1,ANTONIA CASTREJON,ILLUSIONS HAIR DESIGN,3800 W DIVERSEY AVE,CHICAGO,IL,60647,31,999,31-999,25,1010,Limited Business License,NaN,NaN,1000049,RENEW,NaN,2002-06-28T00:00:00.000,2002-08-13T00:00:00.000,N,2002-08-16T00:00:00.000,2003-08-15T00:00:00.000,2002-08-13T00:00:00.000,2002-08-14T00:00:00.000,AAI,NaN,NaN,41.931960,-87.722150,"\n, \n(41.931960332638006, -87.72215036594574)"
4,1000049-20030516,1342680,10141,2,"PEPE""S RETAIL MEATS, INC.",PEREZ MEXICAN FOOD,853-855 W RANDOLPH ST 1ST,CHICAGO,IL,60607,27,1,27-1,12,1006,Retail Food Establishment,775.0,Retail Sales of Perishable Foods,1000049,RENEW,NaN,2003-03-25T00:00:00.000,2003-04-17T00:00:00.000,N,2003-05-16T00:00:00.000,2004-05-15T00:00:00.000,2003-04-17T00:00:00.000,2003-04-18T00:00:00.000,AAI,NaN,NaN,41.884261,-87.649534,"\n, \n(41.88426142200001, -87.6495341312589)"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,1000201-20060216,1663561,200313,1,"KAPPIL STORE, INC",KAPPIL STORE,5952 W DIVERSEY AVE,CHICAGO,IL,60639,30,12,30-12,25,1781,Tobacco,763.0,Retail Sale of Tobacco,1000201,RENEW,NaN,2005-12-21T00:00:00.000,2006-02-09T00:00:00.000,N,2006-02-16T00:00:00.000,2007-02-15T00:00:00.000,2006-02-09T00:00:00.000,2006-02-10T00:00:00.000,AAI,NaN,NaN,41.931252,-87.775672,"\n, \n(41.931252123128445, -87.7756722749983)"
96,1000-20130516,2241648,24139,1,600 SOUTH CICERO INC,BELMONTE LIQUORS,600 S CICERO AVE 1ST,CHICAGO,IL,60644,29,46,29-46,15,1474,Package Goods,774.0,Retail Sales of Packaged Liquor,1000,RENEW,NaN,2013-03-15T00:00:00.000,2013-03-31T00:00:00.000,N,2013-05-16T00:00:00.000,2015-05-15T00:00:00.000,2013-03-31T00:00:00.000,2013-04-01T00:00:00.000,AAI,NaN,NaN,41.873095,-87.745144,"\n, \n(41.873094832519136, -87.74514386884542)"
97,1000-20150516,2386960,24139,1,600 SOUTH CICERO INC,BELMONTE LIQUORS,600 S CICERO AVE 1ST,CHICAGO,IL,60644,29,46,29-46,15,1474,Package Goods,774.0,Retail Sales of Packaged Liquor,1000,RENEW,NaN,2015-03-15T00:00:00.000,2015-04-05T00:00:00.000,N,2015-05-16T00:00:00.000,2017-05-15T00:00:00.000,2015-04-05T00:00:00.000,2015-04-06T00:00:00.000,AAI,NaN,NaN,41.873095,-87.745144,"\n, \n(41.8730948

**Tip:** Use multiple cursors in VS Code to easily edit many lines at the same time (<https://code.visualstudio.com/docs/getstarted/tips-and-tricks#_column-box-selection>).

In [5]:
business_license_schema = pa.DataFrameSchema({
    "id": pa.Column(str, coerce=True, unique=True),
    "license_id": pa.Column(str, coerce=True),
    "account_number": pa.Column(str, coerce=True),
    "site_number": pa.Column(str, coerce=True),
    "legal_name": pa.Column(str, coerce=True),
    "doing_business_as_name": pa.Column(str, coerce=True),
    "address": pa.Column(str, coerce=True),
    "city": pa.Column(str, coerce=True, checks=[
        pa.Check.eq("CHICAGO")
    ]),
    "state": pa.Column(str, coerce=True, checks=[
        pa.Check.eq("IL")
    ]),
    "zip_code": pa.Column(str, coerce=True, checks=[
        pa.Check(lambda x: x.str.match(r'^\d{5}$').all())
    ]),
    "ward": pa.Column(str, coerce=True),
    "precinct": pa.Column(str, coerce=True),
    "ward_precinct": pa.Column(str, coerce=True),
    "police_district": pa.Column(pa.Category, coerce=True),
    "license_code": pa.Column(pa.Category, coerce=True),
    "license_description": pa.Column(str, coerce=True),
    "business_activity_id": pa.Column(str, coerce=True, nullable=True),
    "business_activity": pa.Column(pa.Category, coerce=True, nullable=True),
    "license_number": pa.Column(str, coerce=True),
    "application_type": pa.Column(pa.Category, coerce=True),
    "application_created_date": pa.Column(str, coerce=True, nullable=True),
    "application_requirements_complete": pa.Column(pa.DateTime, coerce=True),
    "payment_date": pa.Column(pa.DateTime, coerce=True),
    "conditional_approval": pa.Column(str, coerce=True),
    "license_start_date": pa.Column(pa.DateTime, coerce=True),
    "expiration_date": pa.Column(pa.DateTime, coerce=True),
    "license_approved_for_issuance": pa.Column(str, coerce=True),
    "date_issued": pa.Column(pa.DateTime, coerce=True),
    "license_status": pa.Column(pa.Category, coerce=True),
    "license_status_change_date": pa.Column(pa.DateTime, coerce=True, nullable=True),
    "ssa": pa.Column(str, coerce=True, nullable=True),
    "latitude": pa.Column(pa.Float, coerce=True, nullable=True, checks=[
        pa.Check.between(38, 44)
    ]),
    "longitude": pa.Column(pa.Float, coerce=True, nullable=True, checks=[
        pa.Check.between(-89, -84)
    ]),
    "location": pa.Column(str, coerce=True),
})

business_license_validated = business_license_schema.validate(business_license_raw)
print(business_license_validated.info())
business_license_validated

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 34 columns):
 #   Column                             Non-Null Count  Dtype         
---  ------                             --------------  -----         
 0   id                                 100 non-null    object        
 1   license_id                         100 non-null    object        
 2   account_number                     100 non-null    object        
 3   site_number                        100 non-null    object        
 4   legal_name                         100 non-null    object        
 5   doing_business_as_name             100 non-null    object        
 6   address                            100 non-null    object        
 7   city                               100 non-null    object        
 8   state                              100 non-null    object        
 9   zip_code                           100 non-null    object        
 10  ward                               100 

,id,license_id,account_number,site_number,legal_name,doing_business_as_name,address,city,state,zip_code,ward,precinct,ward_precinct,police_district,license_code,license_description,business_activity_id,business_activity,license_number,application_type,application_created_date,application_requirements_complete,payment_date,conditional_approval,license_start_date,expiration_date,license_approved_for_issuance,date_issued,license_status,license_status_change_date,ssa,latitude,longitude,location
0,1000000-20020221,1000000,200001,1,MARK BOSTON,COLORS IN MOTION,6421 N DAMEN AVE,CHICAGO,IL,60645,50,28,50-28,24,1011,Home Repair,NaN,NaN,1000000,ISSUE,2000-06-19T00:00:00.000,2002-02-15,2002-02-15,N,2002-02-21,2002-11-15,2002-02-21T00:00:00.000,2002-02-22,AAI,NaT,NaN,41.998514,-87.680011,"\n, \n(41.99851437112669, -87.68001090539342)"
1,1000049-20010816,1162772,200068,1,ANTONIA CASTREJON,ILLUSIONS HAIR DESIGN,3800 W DIVERSEY AVE,CHICAGO,IL,60647,31,999,31-999,25,1010,Limited Business License,NaN,NaN,1000049,RENEW,NaN,2001-06-25,2001-08-20,N,2001-08-16,2002-08-15,2001-08-20T00:00:00.000,2002-04-30,AAI,NaT,NaN,41.931960,-87.722150,"\n, \n(41.931960332638006, -87.72215036594574)"
2,1000049-20020516,1233615,10141,2,"PEPE""S RETAIL MEATS, INC.",PEREZ MEXICAN FOOD,853-855 W RANDOLPH ST 1ST,CHICAGO,IL,60607,27,1,27-1,12,1006,Retail Food Establishment,775.0,Retail Sales of Perishable Foods,1000049,RENEW,NaN,2002-03-27,2002-04-17,N,2002-05-16,2003-05-15,2002-04-17T00:00:00.000,2002-04-18,AAI,NaT,NaN,41.884261,-87.649534,"\n, \n(41.88426142200001, -87.6495341312589)"
3,1000049-20020816,1265665,200068,1,ANTONIA CASTREJON,ILLUSIONS HAIR DESIGN,3800 W DIVERSEY AVE,CHICAGO,IL,60647,31,999,31-999,25,1010,Limited Business License,NaN,NaN,1000049,RENEW,NaN,2002-06-28,2002-08-13,N,2002-08-16,2003-08-15,2002-08-13T00:00:00.000,2002-08-14,AAI,NaT,NaN,41.931960,-87.722150,"\n, \n(41.931960332638006, -87.72215036594574)"
4,1000049-20030516,1342680,10141,2,"PEPE""S RETAIL MEATS, INC.",PEREZ MEXICAN FOOD,853-855 W RANDOLPH ST 1ST,CHICAGO,IL,60607,27,1,27-1,12,1006,Retail Food Establishment,775.0,Retail Sales of Perishable Foods,1000049,RENEW,NaN,2003-03-25,2003-04-17,N,2003-05-16,2004-05-15,2003-04-17T00:00:00.000,2003-04-18,AAI,NaT,NaN,41.884261,-87.649534,"\n, \n(41.88426142200001, -87.6495341312589)"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,1000201-20060216,1663561,200313,1,"KAPPIL STORE, INC",KAPPIL STORE,5952 W DIVERSEY AVE,CHICAGO,IL,60639,30,12,30-12,25,1781,Tobacco,763.0,Retail Sale of Tobacco,1000201,RENEW,NaN,2005-12-21,2006-02-09,N,2006-02-16,2007-02-15,2006-02-09T00:00:00.000,2006-02-10,AAI,NaT,NaN,41.931252,-87.775672,"\n, \n(41.931252123128445, -87.7756722749983)"
96,1000-20130516,2241648,24139,1,600 SOUTH CICERO INC,BELMONTE LIQUORS,600 S CICERO AVE 1ST,CHICAGO,IL,60644,29,46,29-46,15,1474,Package Goods,774.0,Retail Sales of Packaged Liquor,1000,RENEW,NaN,2013-03-15,2013-03-31,N,2013-05-16,2015-05-15,2013-03-31T00:00:00.000,2013-04-01,AAI,NaT,NaN,41.873095,-87.745144,"\n, \n(41.873094832519136, -87.74514386884542)"
97,1000-20150516,2386960,24139,1,600 SOUTH CICERO INC,BELMONTE LIQUORS,600 S CICERO AVE 1ST,CHICAGO,IL,60644,29,46,29-46,15,1474,Package Goods,774.0,Retail Sales of Packaged Liquor,1000,RENEW,NaN,2015-03-15,2015-04-05,N,2015-05-16,2017-05-15,2015-04-05T00:00:00.000,2015-04-06,AAI,NaT,NaN,41.873095,-87.745144,"\n, \n(41.873094832519136, -87.74514386884542)"
98,1000-20170516,2518597,24139,1,600 SOUTH CICERO INC,BELMONTE LIQUORS,600 S CICERO AVE 1ST,CHICAGO,IL,60644,29,46,29-46,15,1474,Package Goods,774.0,Retail Sales of Packaged Liquor,1000,RENEW,NaN,2017-03-15,2017-03-28,N,2017-05-16,2019-05-15,2017-03-28T00:00:00.000,2017-03-29,AAI,NaT,NaN,41.873095,-87.745144,"\n, \n(41.873094832519136, -87.74514386884542)"


In [6]:
# Pin the data to Connect
pin_name = f"{user_name}/chicago-business-license-data-validated"
board.pin_write(
    business_license_validated, 
    name=pin_name, 
    type="csv",
    versioned=True,
    title="City of Chicago - Business License Data (VALIDATED)"
)

Writing pin:
Name: 'sam.edwardes/chicago-business-license-data-validated'
Version: 20230613T115441Z-d3571


Meta(title='City of Chicago - Business License Data (VALIDATED)', description=None, created='20230613T115441Z', pin_hash='d35713a82391f09b', file='chicago-business-license-data-validated.csv', file_size=35718, type='csv', api_version=1, version=VersionRaw(version='75842'), tags=None, name='sam.edwardes/chicago-business-license-data-validated', user={}, local={})

## Data set (2): Food inspections

<https://data.cityofchicago.org/Health-Human-Services/Food-Inspections/4ijn-s7e5>

In [7]:
pin_name = f"{user_name}/chicago-food-inspection-data"
food_inspection_raw = board.pin_read(pin_name)
food_inspection_raw

,inspection_id,dba_name,aka_name,license_,facility_type,risk,address,city,state,zip,inspection_date,inspection_type,results,violations,latitude,longitude,location
0,67757,DUNKIN DONUTS/BASKIN-ROBBINS,DUNKIN DONUTS/BASKIN-ROBBINS,1380279,Restaurant,Risk 2 (Medium),100 W RANDOLPH ST,CHICAGO,IL,60601,2010-01-04T00:00:00.000,Tag Removal,Pass,NaN,41.884586,-87.631010,"(41.88458626715456, -87.63101044588599)"
1,67733,WOLCOTT'S,TROQUET,1992040,Restaurant,Risk 1 (High),1834 W MONTROSE AVE,CHICAGO,IL,60613,2010-01-04T00:00:00.000,License Re-Inspection,Pass,NaN,41.961606,-87.675967,"(41.961605669949854, -87.67596676683779)"
2,104236,TEMPO CAFE,TEMPO CAFE,80916,Restaurant,Risk 1 (High),6 E CHESTNUT ST,CHICAGO,IL,60611,2010-01-04T00:00:00.000,Canvass,Fail,18. NO EVIDENCE OF RODENT OR INSECT OUTER OPEN...,41.898431,-87.628009,"(41.89843137207629, -87.6280091630558)"
3,52234,Cafe 608,Cafe 608,2013328,Restaurant,Risk 1 (High),608 W BARRY AVE,CHICAGO,IL,60657,2010-01-04T00:00:00.000,License Re-Inspection,Pass,NaN,41.938007,-87.644755,"(41.938006880423615, -87.6447545707008)"
4,70269,mr.daniel's,mr.daniel's,1899292,Restaurant,Risk 1 (High),5645 W BELMONT AVE,CHICAGO,IL,60634,2010-01-04T00:00:00.000,License Re-Inspection,Pass,NaN,41.938443,-87.768318,"(41.93844282365204, -87.76831838068422)"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,58229,FILTER COFFEE LOUNGE,FILTER,2013495,Restaurant,Risk 1 (High),1373 N MILWAUKEE AVE,CHICAGO,IL,60622,2010-01-06T00:00:00.000,Consultation,Fail,NaN,41.906868,-87.671749,"(41.90686820755232, -87.67174891786233)"
96,70275,THE GREAT AMERICAN BAGEL,THE GREAT AMERICAN BAGEL,1488510,Restaurant,Risk 1 (High),11601 W TOUHY AVE,CHICAGO,IL,60666,2010-01-06T00:00:00.000,Out of Business,Pass,NaN,42.008536,-87.914428,"(42.008536400868735, -87.91442843927047)"
97,80210,S L PANTRY & GRILL,NaN,2013420,Grocery Store,Risk 1 (High),4759 N SHERIDAN RD,CHICAGO,IL,60640,2010-01-06T00:00:00.000,License Re-Inspection,Pass,NaN,41.969047,-87.654674,"(41.969046674717234, -87.65467405897606)"
98,134246,PANNENKOEKEN CAFE2,PANNENKOEKEN CAFE,1986032,Restaurant,Risk 1 (High),2257 W NORTH AVE,CHICAGO,IL,60647,2010-01-06T00:00:00.000,Complaint,Pass,33. FOOD AND NON-FOOD CONTACT EQUIPMENT UTENSI...,41.910250,-87.684634,"(41.91024956670776, -87.68463448701652)"


In [8]:
food_inspection_schema = pa.DataFrameSchema({
    "inspection_id": pa.Column(str, coerce=True, unique=True),
    "dba_name": pa.Column(str, coerce=True),
    "aka_name": pa.Column(str, coerce=True, nullable=True),
    "license_": pa.Column(str, coerce=True),
    "facility_type": pa.Column(pa.Category, coerce=True, nullable=True),
    "risk": pa.Column(str, coerce=True),
    "address": pa.Column(str, coerce=True),
    "city": pa.Column(str, coerce=True, checks=[
        pa.Check.eq("CHICAGO")
    ]),
    "state": pa.Column(str, coerce=True),
    "zip": pa.Column(str, coerce=True),
    "inspection_date": pa.Column(pa.DateTime, coerce=True),
    "inspection_type": pa.Column(str, coerce=True),
    "results": pa.Column(str, coerce=True),
    "violations": pa.Column(str, coerce=True, nullable=True),
    "latitude": pa.Column(pa.Float, coerce=True, nullable=True, checks=[
        pa.Check.between(38, 44)
    ]),
    "longitude": pa.Column(pa.Float, coerce=True, nullable=True, checks=[
        pa.Check.between(-89, -84)
    ]),
    "location": pa.Column(str, coerce=True, nullable=True),
})

food_inspection_validated = food_inspection_schema.validate(food_inspection_raw)
print(food_inspection_validated.info())
food_inspection_validated

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 17 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   inspection_id    100 non-null    object        
 1   dba_name         100 non-null    object        
 2   aka_name         94 non-null     object        
 3   license_         100 non-null    object        
 4   facility_type    95 non-null     category      
 5   risk             100 non-null    object        
 6   address          100 non-null    object        
 7   city             100 non-null    object        
 8   state            100 non-null    object        
 9   zip              100 non-null    object        
 10  inspection_date  100 non-null    datetime64[ns]
 11  inspection_type  100 non-null    object        
 12  results          100 non-null    object        
 13  violations       65 non-null     object        
 14  latitude         99 non-null     float64   

,inspection_id,dba_name,aka_name,license_,facility_type,risk,address,city,state,zip,inspection_date,inspection_type,results,violations,latitude,longitude,location
0,67757,DUNKIN DONUTS/BASKIN-ROBBINS,DUNKIN DONUTS/BASKIN-ROBBINS,1380279,Restaurant,Risk 2 (Medium),100 W RANDOLPH ST,CHICAGO,IL,60601,2010-01-04,Tag Removal,Pass,NaN,41.884586,-87.631010,"(41.88458626715456, -87.63101044588599)"
1,67733,WOLCOTT'S,TROQUET,1992040,Restaurant,Risk 1 (High),1834 W MONTROSE AVE,CHICAGO,IL,60613,2010-01-04,License Re-Inspection,Pass,NaN,41.961606,-87.675967,"(41.961605669949854, -87.67596676683779)"
2,104236,TEMPO CAFE,TEMPO CAFE,80916,Restaurant,Risk 1 (High),6 E CHESTNUT ST,CHICAGO,IL,60611,2010-01-04,Canvass,Fail,18. NO EVIDENCE OF RODENT OR INSECT OUTER OPEN...,41.898431,-87.628009,"(41.89843137207629, -87.6280091630558)"
3,52234,Cafe 608,Cafe 608,2013328,Restaurant,Risk 1 (High),608 W BARRY AVE,CHICAGO,IL,60657,2010-01-04,License Re-Inspection,Pass,NaN,41.938007,-87.644755,"(41.938006880423615, -87.6447545707008)"
4,70269,mr.daniel's,mr.daniel's,1899292,Restaurant,Risk 1 (High),5645 W BELMONT AVE,CHICAGO,IL,60634,2010-01-04,License Re-Inspection,Pass,NaN,41.938443,-87.768318,"(41.93844282365204, -87.76831838068422)"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,58229,FILTER COFFEE LOUNGE,FILTER,2013495,Restaurant,Risk 1 (High),1373 N MILWAUKEE AVE,CHICAGO,IL,60622,2010-01-06,Consultation,Fail,NaN,41.906868,-87.671749,"(41.90686820755232, -87.67174891786233)"
96,70275,THE GREAT AMERICAN BAGEL,THE GREAT AMERICAN BAGEL,1488510,Restaurant,Risk 1 (High),11601 W TOUHY AVE,CHICAGO,IL,60666,2010-01-06,Out of Business,Pass,NaN,42.008536,-87.914428,"(42.008536400868735, -87.91442843927047)"
97,80210,S L PANTRY & GRILL,NaN,2013420,Grocery Store,Risk 1 (High),4759 N SHERIDAN RD,CHICAGO,IL,60640,2010-01-06,License Re-Inspection,Pass,NaN,41.969047,-87.654674,"(41.969046674717234, -87.65467405897606)"
98,134246,PANNENKOEKEN CAFE2,PANNENKOEKEN CAFE,1986032,Restaurant,Risk 1 (High),2257 W NORTH AVE,CHICAGO,IL,60647,2010-01-06,Complaint,Pass,33. FOOD AND NON-FOOD CONTACT EQUIPMENT UTENSI...,41.910250,-87.684634,"(41.91024956670776, -87.68463448701652)"


In [9]:
# Pin the data to Connect
pin_name = f"{user_name}/chicago-food-inspection-data-validated"
board.pin_write(
    food_inspection_validated, 
    name=pin_name, 
    type="csv",
    versioned=True,
    title="City of Chicago - Food Inspection Data (VALIDATED)"
)

Writing pin:
Name: 'sam.edwardes/chicago-food-inspection-data-validated'
Version: 20230613T115450Z-56fc5


Meta(title='City of Chicago - Food Inspection Data (VALIDATED)', description=None, created='20230613T115450Z', pin_hash='56fc51d91b5c6806', file='chicago-food-inspection-data-validated.csv', file_size=144030, type='csv', api_version=1, version=VersionRaw(version='75843'), tags=None, name='sam.edwardes/chicago-food-inspection-data-validated', user={}, local={})

# Tmp

In [10]:
# profile = ProfileReport(business_license_raw, title="Profiling Report")
# profile